TERMINOLOGY MINING: This is the basic conversation to get a list of terminology that the user needs to know about to be able to understand the goal.

The reason for doing it this way, is because: when you ask the gpt "give terminology related to 'machine learning'" that the person should know, when for instance the person doesn't know some basic terminology, it still gives only quite advanced terms. Often, when learning something advanced without knowing the basics, is incredibly difficult. While if you have mastered the basics, the advanced level is quite easy. This way, it's possible to also "mine" these basic terms.

However, a question that can be asked is: when you want to explain machine learning, it's a quite broad term. And how deep do you want to explain it? A 'broad' definition is 'Machine learning is a subset of artificial intelligence (AI) that focuses on building systems that can learn from and make decisions based on data.'. In this definition they completely don't talk about the mathematics behind it, while the mathematics behind it is quite big and gives a completely another level of understanding.

SideNote (TODO): However, the step of searching for terminology that the LLM doesn't know yet doesn't have to be done by a LLM. This can be done a lot more quickly and cheaply with normal code. (TODO). The part where the LLM is useful, is to create automatic definitions of these new terminologies to search for new terminologies.

In [3]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    file_location=".",
    filter_dict={
        "model": ["gpt-4", "gpt4", "gpt-4-32k"],
    },
)

print("LLM models: ", [config_list[i]["model"] for i in range(len(config_list))])

llm_config = {
    "timeout": 60,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

# autogen.ChatCompletion.start_logging()
termination_msg = lambda x: isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

LLM models:  ['gpt-4-32k', 'gpt-4']


In [6]:
system_message = """We are playing a simulation game. You are a helper searching for terminology not in your terminology list.
You are only allowed to speak with the instructions given below and nothing else. 

You have limited terminology knowledge, but you can learn new terminology. 
Imagine know have a TERMINOLOGY-LIST of a typical 16 year old teenager as database.
Below you will get extra terminology you know in addition to the terminology of a typical 16 year old teenager.
If words are not existent in the TERMINOLOGY-LIST, you answer with "DONT UNDERSTAND:" followed with the list. 
and give a list of words why you are answering with 'DONT UNDERSTAND:'.
If a verb is in the TERMINOLOGY-LIST, you also know the conjugations.

IF you get an explanation of the new terms, analyze the text for new terms that are not in your TERMINOLOGY-LIST.
IF all terms in the text are in the TERMINOLOGY-LIST, list all terms and end with TERMINATE as last word.

Extra words you know: 
"""

# 1. create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant", 
    system_message=system_message,
    llm_config={
        "timeout": 600,
        "seed": 42,
        "config_list": config_list,
    },
    is_termination_msg=termination_msg
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = autogen.AssistantAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    system_message="""You are a helpful AI Assistant. If the assistant answers with 'DONT UNDERSTAND:' and a list of words,
    THEN say: "Ok, I'll explain it to you:", 
    THEN you should answer with explanations for those words in less than 2 sentences.
    At the end, always ask the question: Are there any terms in this explanation not in your terminology list?
    If the assistant replies with TERMINATE then TERMINATE.""",
    max_consecutive_auto_reply=10,
    is_termination_msg=termination_msg,
    llm_config={
        "timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)
user_message = """Demonstrate knowledge of machine learning terminology such as likelihood function, maximum likelihood, Bayesian inference, feed-forward, convolutional and Transformer neural networks, and error back propagation."""
# the assistant receives a message from the user_proxy, which contains the task description
user_proxy.initiate_chat(
    assistant,
    message=user_message,
)

user_proxy (to assistant):

Demonstrate knowledge of machine learning terminology such as likelihood function, maximum likelihood, Bayesian inference, feed-forward, convolutional and Transformer neural networks, and error back propagation.

--------------------------------------------------------------------------------
assistant (to user_proxy):

DONT UNDERSTAND: likelihood function, maximum likelihood, Bayesian inference, feed-forward, convolutional, Transformer neural networks, error back propagation.

--------------------------------------------------------------------------------
user_proxy (to assistant):

Ok, I'll explain it to you:

1. Likelihood Function: It's a function that shows how 'likely' our data samples are, given certain parameters.
2. Maximum Likelihood: It's a method to estimate the parameters of a model that maximizes the likelihood function.
3. Bayesian Inference: It's a statistical method that updates the probability of a hypothesis as more evidence/ data becomes

AttributeError: 'NoneType' object has no attribute 'model_dump'